## Applying machine learning to number theory problem - Naive Bayes

Sato-tate group classification problem:

Given normalized Frobenius traces, is it possible to 
- determine whether Sato-Tate group  is USp(4)?
- Extended problem: Classifying specific type of Sato-Tate group in the LMF (6 classes)

#### Some applicable machine learning techniques
- Multi-layer perceptron
- Naive Bayes classification

In [1]:
import numpy as np
import json
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split

from math import sqrt
from math import exp
from math import pi
import statistics

In [2]:
### preprocessing and Spliting data
def split_data_train_test(data, long_input=True, normalized_data=True, test_size=0.2):
    datapoints = [point.split(':') for point in data]
    labels_data = [int(point[0]) for point in datapoints]
    if long_input:
        input_data = [json.loads(point[3]) for point in datapoints]
    else:
        input_data = [json.loads(point[1]) for point in datapoints]
    input_data_pd = pd.DataFrame(input_data)
    if not long_input:
        input_data_pd[6] = input_data_pd[6].fillna(0)

    if normalized_data:
        input_data_pd = preprocessing.normalize(input_data_pd)
        input_data_pd = pd.DataFrame(input_data_pd, columns=input_data_pd.columns)
    
    X = input_data_pd.to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(X, labels_data, test_size=0.2)
    return X_train, X_test, y_train, y_test

In [5]:
with open('./g2st.txt') as f:
    data = [l for l in f.readlines()]

In [6]:
X_train, X_test, y_train, y_test = split_data_train_test(data, normalized_data=False)
train_dat = np.concatenate([X_train, np.array([y_train]).reshape(X_train.shape[0],1)],axis=1)
test_dat = np.concatenate([X_test, np.array([y_test]).reshape(X_test.shape[0],1)],axis=1)

### Gaussian Naive Bayes
Given a point $X_i = [x_1 \cdot x_n]$ in dataset $X$ where each datapoint $X_i$ belongs to class $c \in C$. . Determine the label of $X_i$
- $P(C_j|X_i) \propto P(x_1|C_j)\cdot P(x_i|C_j)\cdots P(x_n|C_j) \cdot P(C_j)$
- $argmax([P(C_1|X_i), \cdots P(C_m|X_i)])$

### Using naive bayes pre-built library function

In [7]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
acc = (y_test == y_pred).sum()/X_test.shape[0]
print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (y_test != y_pred).sum()))
print(f"accuracy: {acc}")

Number of mislabeled points out of a total 200000 points : 2591
accuracy: 0.987045


In [8]:
# Naive Bayes Algorithm
def naive_bayes(train, test):
    summarize = summarize_by_class(train)
    predictions = list()
    for row in test:
        output = predict(summarize, row)
        predictions.append(output)
    return(predictions)

#########################
## summarizing dataset ##
#########################

# for each class, get mean, stdev of each feature, size of that class
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries

def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if (class_value not in separated):
            separated[class_value] = list()
        separated[class_value].append(vector[:-1])
    return separated
def summarize_dataset(dataset):
    summaries = [(statistics.mean(column), statistics.stdev(column), len(column)) for column in zip(*dataset)]
    return summaries


In [9]:
################################################
## generating prediction based on Gaussian NB ##
################################################

# Calculate the Gaussian probability distribution function for x (prob density func with x)
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        # calculate the prior (# instance of class i / dataset size)
        probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
        # for each feature x_i, calculate likelihood & update the current probability (multiply with likelihood)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

# Predict the class for a given row
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

# def accuracy_metric(actual, predicted):
#     correct = 0
#     for i in range(len(actual)):
#         if actual[i] == predicted[i]:
#             correct += 1
#     return correct / float(len(actual)) * 100.0

In [10]:
predicted = naive_bayes(train_dat, test_dat)

In [11]:
acc = [0 if i != j else 1 for i,j in zip(y_test, predicted)].sum()/X_test.shape[0]
print(acc)
#accuracy_metric(y_test, predicted)

AttributeError: 'bool' object has no attribute 'sum'

In [14]:
predicted[:10]

False

In [12]:
### preprocessing and Spliting data
def split_data_train_test_mult_class(data, long_input=True, normalized_data=True, test_size=0.2):
    datapoints = [point.split(':') for point in data]
    labels_data = [(ord(point[0]) - ord('A')) for point in datapoints]
    if long_input:
        input_data = [json.loads(point[4]) for point in datapoints]
    else:
        input_data = [json.loads(point[2]) for point in datapoints]
    input_data_pd = pd.DataFrame(input_data)
    if not long_input:
        input_data_pd[6] = input_data_pd[6].fillna(0)

    if normalized_data:
        input_data_pd = preprocessing.normalize(input_data_pd)
        input_data_pd = pd.DataFrame(input_data_pd, columns=input_data_pd.columns)
    
    X = input_data_pd.to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(X, labels_data, test_size=0.2)
    return X_train, X_test, y_train, y_test


In [11]:
with open('./g2st2.txt') as f:
    data = [l for l in f.readlines()]

In [13]:
X_train, X_test, y_train, y_test = split_data_train_test_mult_class(data, normalized_data=False)
train_dat = np.concatenate([X_train, np.array([y_train]).reshape(X_train.shape[0],1)],axis=1)
test_dat = np.concatenate([X_test, np.array([y_test]).reshape(X_test.shape[0],1)],axis=1)

In [16]:
predicted = naive_bayes(train_dat, test_dat)

In [17]:
accuracy_metric(y_test, predicted)

96.265

#### Bibliography

- Application of naive bayes to Sato-Tate problem taken from the following paper: He, YH, Lee, KH, Oliver, T. Machine Learning the Sato-Tate Conjecture. (2020) https://arxiv.org/abs/2010.01213

- Implementation of naive bayes is inspired by https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/ 